### Structured Output Parser - Python Object

In [1]:
import os

from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional
from langchain.chains.openai_functions import create_structured_output_chain

In [2]:
load_dotenv(override=True)

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
azure_api_version = os.getenv("AZURE_API_VERSION")

if not azure_endpoint or not azure_api_key or not azure_deployment or not azure_api_version:
    raise ValueError("Azure OpenAI environment variables are not set.")

temperature = 0.7
max_tokens = 1500

llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    azure_deployment=azure_deployment,
    api_version=azure_api_version,
    temperature=temperature,
    max_tokens=max_tokens,
    model="gpt-4o"
)

In [3]:
class Person(BaseModel):
    name: str = Field(..., description="The name of the person")
    age: Optional[int] = Field(None, description="The age of the person")
    occupation: Optional[str] = Field(None, description="The occupation of the person")
    years_of_experience: Optional[int] = Field(None, description="Years of experience in their field")
    skills: Optional[str] = Field(None, description="Skills of the person")

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for extracting structured data from text."),
        ("human",
         "User the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format.")
    ])

chain = create_structured_output_chain(
    llm=llm,
    output_schema=Person,
    prompt=prompt
)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_30100\3978735300.py:9: LangChainDeprecationWarning: The function `create_structured_output_chain` was deprecated in LangChain 0.1.1 and will be removed in 1.0. Use :meth:`~ChatOpenAI.with_structured_output` instead.
  chain = create_structured_output_chain(


In [5]:
request = """
    Ramkumar is 50 Years Old, and he works as a Software Engineer at Google.
    He has 20 years of experience in the field and is skilled in Python, Java, and C++.
"""

response = chain.invoke({"input": request})



In [6]:
print("Response:", response)

person = response["function"]

if isinstance(person, Person):
    print("Extracted Data:")
    print(f"Name: {person.name}")
    print(f"Age: {person.age}")
    print(f"Occupation: {person.occupation}")
    print(f"Years of Experience: {person.years_of_experience}")
    print(f"Skills: {person.skills}")

Response: {'input': '\n    Ramkumar is 50 Years Old, and he works as a Software Engineer at Google.\n    He has 20 years of experience in the field and is skilled in Python, Java, and C++.\n', 'function': Person(name='Ramkumar', age=50, occupation='Software Engineer', years_of_experience=20, skills='Python, Java, and C++')}
Extracted Data:
Name: Ramkumar
Age: 50
Occupation: Software Engineer
Years of Experience: 20
Skills: Python, Java, and C++


In [7]:
from typing import Sequence

class People(BaseModel):
    people: Sequence[Person] = Field(..., description="List of people extracted from the text")

chain = create_structured_output_chain(
    llm=llm,
    output_schema=People,
    prompt=prompt
)

In [8]:
request = """
    Ramkumar is 50 Years Old, and he works as a Software Engineer at Google.
    He has 20 years of experience in the field and is skilled in Python, Java, and C++.
    
    Lakshmi is 41 Years Old, and he works as a Software Engineer at Microsoft.
    He has 20 years of experience in the field and is skilled in Python, Java, and C++.
    
    Renu  is 37 Years Old, and he works as a Software Engineer at Microsoft.
    He has 20 years of experience in the field and is skilled in Python, Java, and C++.
    
    Both of them are skilled LangChain and OpenAI.
    
    People who worked at microsoft have common skills such as C#, .NET, Azure, and SQL Server.
"""

response = chain.invoke({"input": request})

In [9]:
print(response)

{'input': '\n    Ramkumar is 50 Years Old, and he works as a Software Engineer at Google.\n    He has 20 years of experience in the field and is skilled in Python, Java, and C++.\n\n    Lakshmi is 41 Years Old, and he works as a Software Engineer at Microsoft.\n    He has 20 years of experience in the field and is skilled in Python, Java, and C++.\n\n    Renu  is 37 Years Old, and he works as a Software Engineer at Microsoft.\n    He has 20 years of experience in the field and is skilled in Python, Java, and C++.\n\n    Both of them are skilled LangChain and OpenAI.\n\n    People who worked at microsoft have common skills such as C#, .NET, Azure, and SQL Server.\n', 'function': People(people=[Person(name='Ramkumar', age=50, occupation='Software Engineer', years_of_experience=20, skills='Python, Java, C++, LangChain, OpenAI'), Person(name='Lakshmi', age=41, occupation='Software Engineer', years_of_experience=20, skills='Python, Java, C++, LangChain, OpenAI, C#, .NET, Azure, SQL Server')

In [10]:
people = response["function"]

if isinstance(people, People):
    print("Extracted Data:")

    for person in people.people:
        print(f"Name: {person.name}")
        print(f"Age: {person.age}")
        print(f"Occupation: {person.occupation}")
        print(f"Years of Experience: {person.years_of_experience}")
        print(f"Skills: {person.skills}")
        print("-" * 20)
        

Extracted Data:
Name: Ramkumar
Age: 50
Occupation: Software Engineer
Years of Experience: 20
Skills: Python, Java, C++, LangChain, OpenAI
--------------------
Name: Lakshmi
Age: 41
Occupation: Software Engineer
Years of Experience: 20
Skills: Python, Java, C++, LangChain, OpenAI, C#, .NET, Azure, SQL Server
--------------------
Name: Renu
Age: 37
Occupation: Software Engineer
Years of Experience: 20
Skills: Python, Java, C++, LangChain, OpenAI, C#, .NET, Azure, SQL Server
--------------------
